BoojCodeTest

# Imports

In [1]:
import requests

user_agent_url = 'http://syndication.enterprise.websiteidx.com/feeds/BoojCodeTest.xml'
xml_data = requests.get(user_agent_url).content

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd

### Class standardizes xml data acquisition
Source: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

In [3]:
class XML2DataFrame:

    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

xml2df = XML2DataFrame(xml_data)
xml_dataframe = xml2df.process_data()

`.shape` show sus the data we have to work with

In [4]:
xml_dataframe.shape

(219, 84)

In [5]:
xml_dataframe.head()

,AdditionalFeatures,Agent,AlwaysEmailAgent,Appliance,Appliances,ArchitectureStyle,BasicDetails,Bedrooms,BrokerEmail,BrokerPhone,...,Tax,Taxes,ThreeQuarterBathrooms,Title,UnitNumber,UtilitiesIncluded,ViewTypes,VirtualTourUrl,Zip,year-built
0,NaN,\n,0,NaN,NaN,NaN,\n,0,inquire@thepartnerstrust.com,310-858-6800,...,\n\t,\n\t,NaN,0 Castro Peak Mountainway,NaN,\n\t,\n\t,http://www.thepartnerstrust.com/property/42921...,90265,NaN
1,NaN,\n,0,NaN,NaN,NaN,\n,0,inquire@thepartnerstrust.com,310-858-6800,...,\n\t,\n\t,NaN,0 SADDLE PEAK RD,NaN,\n\t,\n\t,http://www.thepartnerstrust.com/property/42922...,90265,NaN
2,NaN,\n,0,RangeOven,\n,Architectural,\n,3,inquire@thepartnerstrust.com,310-858-6800,...,\n\t,\n\t,NaN,21310 PACIFIC COAST HWY,NaN,\n\t,\n\t,http://www.thepartnerstrust.com/property/42922...,90265,NaN
3,NaN,\n,0,RangeOven,\n,Traditional,\n,5,inquire@thepartnerstrust.com,310-858-6800,...,\n\t,\n\t,NaN,23826 MALIBU RD,NaN,\n\t,\n\t,http://www.thepartnerstrust.com/property/42923...,90265,1975
4,NaN,\n,0,RangeOven,\n,Mediterranean,\n,2,inquire@thepartnerstrust.com,310-858-6800,...,\n\t,\n\t,NaN,943 16th ST #5,5,\n\t,\n\t,http://www.thepartnerstrust.com/property/44255...,90403,NaN


### Required Fields

In [6]:
req_fields = ['MlsId',
'MlsName',
'DateListed',
'StreetAddress',
'Price',
'Bedrooms',
'FullBathrooms',
'HalfBathrooms',
'Appliance',
'Appliances',
'Room',
'RoomCount',
'Rooms',
'Description']

### Dataframe only includes the required fields

In [7]:
df_for_csv00 = xml_dataframe[req_fields]
df_for_csv00.head()

,MlsId,MlsName,DateListed,StreetAddress,Price,Bedrooms,FullBathrooms,HalfBathrooms,Appliance,Appliances,Room,RoomCount,Rooms,Description
0,14799273,CLAW,2014-10-03 00:00:00,"23410 Civic Center Way, C1",535000.00,0,NaN,NaN,NaN,NaN,NaN,0,NaN,Enjoy amazing ocean and island views from this...
1,14802845,CLAW,2014-10-17 00:00:00,"23410 Civic Center Way, C1",200000.00,0,NaN,NaN,NaN,NaN,NaN,0,NaN,Spectacular views from this 4+ acre property p...
2,15883387,CLAW,2015-03-03 00:00:00,"23410 Civic Center Way, C1",23500.00,3,4,NaN,RangeOven,\n,bathrooms,2,\n\t,A STUNNING Architectural beautifully designed ...
3,15888095,CLAW,2015-03-18 00:00:00,"23410 Civic Center Way, C1",72500.00,5,4,1,RangeOven,\n,bathrooms,2,\n\t,"July & August not available. ""Captured in Para..."
4,15959941,CLAW,2015-11-18 00:00:00,1333 Montana Ave,230000.00,2,1,NaN,RangeOven,\n,bathrooms,2,\n\t,Due to an overwhelming response we are no long...


In [8]:
df_for_csv00.shape

(219, 14)

### Filter by Description containing the word 'and'

In [9]:
import re

In [10]:
# filter the df based on Description observations containing the word 'and'
df_for_csv000 = df_for_csv00[ df_for_csv00.Description.str.contains('and') ]
df_for_csv000.shape

(211, 14)

So, the process of eliminating rows of data without the word 'and' only dropped 8 rows from our dataset

### Collect just the first 200 characters of Description

In [11]:
new_col = []
for x in df_for_csv000.Description:
    b = x[:200]
    new_col.append(b)

In [12]:
Desc_200 = pd.DataFrame(new_col)
Desc_200.head()

,0
0,Enjoy amazing ocean and island views from this...
1,Spectacular views from this 4+ acre property p...
2,A STUNNING Architectural beautifully designed ...
3,"July & August not available. ""Captured in Para..."
4,Due to an overwhelming response we are no long...


### Rename the awkwardly titled Description column

In [13]:
Desc_200.rename(columns={0: 'Desc'}, inplace=True)

In [14]:
Desc_200.columns

Index([u'Desc'], dtype='object')

### Concatenate the dataframes

In [15]:
df_for_csv01 = pd.concat([df_for_csv00, Desc_200], axis=1)

### Drop the extraneous column

In [16]:
df_for_csv02 = df_for_csv01.drop(['Description'], axis=1)

#### Sanity check

In [17]:
df_for_csv02.head()

,MlsId,MlsName,DateListed,StreetAddress,Price,Bedrooms,FullBathrooms,HalfBathrooms,Appliance,Appliances,Room,RoomCount,Rooms,Desc
0,14799273,CLAW,2014-10-03 00:00:00,"23410 Civic Center Way, C1",535000.00,0,NaN,NaN,NaN,NaN,NaN,0,NaN,Enjoy amazing ocean and island views from this...
1,14802845,CLAW,2014-10-17 00:00:00,"23410 Civic Center Way, C1",200000.00,0,NaN,NaN,NaN,NaN,NaN,0,NaN,Spectacular views from this 4+ acre property p...
2,15883387,CLAW,2015-03-03 00:00:00,"23410 Civic Center Way, C1",23500.00,3,4,NaN,RangeOven,\n,bathrooms,2,\n\t,A STUNNING Architectural beautifully designed ...
3,15888095,CLAW,2015-03-18 00:00:00,"23410 Civic Center Way, C1",72500.00,5,4,1,RangeOven,\n,bathrooms,2,\n\t,"July & August not available. ""Captured in Para..."
4,15959941,CLAW,2015-11-18 00:00:00,1333 Montana Ave,230000.00,2,1,NaN,RangeOven,\n,bathrooms,2,\n\t,Due to an overwhelming response we are no long...


### Sort dataframe by Date and check datatype

In [18]:
df_for_csv03 = df_for_csv02.sort_values(by='DateListed', ascending=False)

In [19]:
type(df_for_csv03.DateListed[0])

str

### Convert the `DateListed` column to pd.datetime datatype, which is more useful

In [20]:
df_for_csv03['DateAsDate'] = pd.to_datetime(df_for_csv03.DateListed)


In [21]:
df_for_csv03['DateAsDate'].head()

214   2016-11-18 07:45:06
155   2016-11-18 00:00:00
212   2016-11-18 00:00:00
156   2016-11-18 00:00:00
154   2016-11-17 00:00:00
Name: DateAsDate, dtype: datetime64[ns]

In [22]:
df_for_csv03.head()

,MlsId,MlsName,DateListed,StreetAddress,Price,Bedrooms,FullBathrooms,HalfBathrooms,Appliance,Appliances,Room,RoomCount,Rooms,Desc,DateAsDate
214,316010702,ITECH,2016-11-18 07:45:06,"9378 Wilshire Boulevard, Suite 200",1025000.00,4,1,NaN,Built-In Gas,\n,bathrooms,2,\n\t,NaN,2016-11-18 07:45:06
155,16181224,CLAW,2016-11-18 00:00:00,"11726 San Vicente Blvd, Suite 350",839000.00,2,2,1,RangeOven,\n,bathrooms,2,\n\t,Meticulously maintained light-filled townhouse...,2016-11-18 00:00:00
212,316010634,ITECH,2016-11-18 00:00:00,"9378 Wilshire Boulevard, Suite 200",785000.00,2,3,NaN,Range Hood,\n,bathrooms,2,\n\t,NaN,2016-11-18 00:00:00
156,16181228,CLAW,2016-11-18 00:00:00,"23410 Civic Center Way, C1",7488000.00,4,4,1,RangeOven,\n,bathrooms,2,\n\t,Located on a quiet street in the desirable San...,2016-11-18 00:00:00
154,16180298,CLAW,2016-11-17 00:00:00,"11726 San Vicente Blvd, Suite 350",1125000.00,3,2,NaN,Self Cleaning Oven,\n,bathrooms,2,\n\t,Absolutely exquisite Spanish Colonial Revival ...,2016-11-17 00:00:00


In [23]:
df_for_csv03.set_index('DateAsDate', inplace=True)

In [24]:
df_for_csv03['Year'] = df_for_csv03.index.year
df_for_csv03['Month'] = df_for_csv03.index.month

In [25]:
df_for_csv03['2016']

,MlsId,MlsName,DateListed,StreetAddress,Price,Bedrooms,FullBathrooms,HalfBathrooms,Appliance,Appliances,Room,RoomCount,Rooms,Desc,Year,Month
DateAsDate,,,,,,,,,,,,,,,,
2016-11-18 07:45:06,316010702,ITECH,2016-11-18 07:45:06,"9378 Wilshire Boulevard, Suite 200",1025000.00,4,1,NaN,Built-In Gas,\n,bathrooms,2,\n\t,NaN,2016,11
2016-11-18 00:00:00,16181224,CLAW,2016-11-18 00:00:00,"11726 San Vicente Blvd, Suite 350",839000.00,2,2,1,RangeOven,\n,bathrooms,2,\n\t,Meticulously maintained light-filled townhouse...,2016,11
2016-11-18 00:00:00,316010634,ITECH,2016-11-18 00:00:00,"9378 Wilshire Boulevard, Suite 200",785000.00,2,3,NaN,Range Hood,\n,bathrooms,2,\n\t,NaN,2016,11
2016-11-18 00:00:00,16181228,CLAW,2016-11-18 00:00:00,"23410 Civic Center Way, C1",7488000.00,4,4,1,RangeOven,\n,bathrooms,2,\n\t,Located on a quiet street in the desirable San...,2016,11
2016-11-17 00:00:00,16180298,CLAW,2016-11-17 00:00:00,"11726 San Vicente Blvd, Suite 350",1125000.00,3,2,NaN,Self Cleaning Oven,\n,bathrooms,2,\n\t,Absolutely exquisite Spanish Colonial Revival ...,2016,11
2016-11-17 00:00:00,316010650,ITECH,2016-11-17 00:00:00,964 Foothill Boulevard,1299000.00,4,3,1,Built-In Gas,\n,bathrooms,2,\n\t,NaN,2016,11
2016-11-16 00:00:00,16180182,CLAW,2016-11-16 00:00:00,"9378 Wilshire Boulevard, Suite 200",1875000.00,3,3,1,RangeOven,\n,bathrooms,2,\n\t,Unbelievable custom home in the hills of La Ca...,2016,11
2016-11-14 00:00:00,316010536,ITECH,2016-11-14 00:00:00,964 Foothill Boulevard,2600000.00,5,4,NaN,Built-In BBQ,\n,bathrooms,2,\n\t,NaN,2016,11
2016-11-14 00:00:00,16179602,CLAW,2016-11-14 00:00:00,"11726 San Vicente Blvd, Suite 350",3986000.00,3,1,NaN,RangeOven,\n,bathrooms,2,\n\t,"House was taken down to the studs, has no plum...",2016,11


In [26]:
df_for_csv03.shape

(219, 16)

In [27]:
df_for_csv03['2016'].shape

(209, 16)

### Re-title the df with only 2016 values

In [28]:
df_for_csv04 = df_for_csv03['2016']

### Appliances... all sub-nodes comma joined

In [29]:
# this is a candidate for sub-nodes, but is redundant
df_for_csv04.Appliances.value_counts()

\n            156
Name: Appliances, dtype: int64

In [30]:
# this is a clear candidate for sub-nodes, and has relevant information
df_for_csv04.Appliance.value_counts()

RangeOven                     100
Oven                           12
Built-In Gas                    7
Oven-Gas                        6
Microwave                       6
Range Hood                      6
Built-Ins                       6
Built-In BBQ                    3
Free Standing Gas               2
Cooktop - Gas                   2
Gas                             2
Double Oven                     2
Built-In And Free Standing      1
Self Cleaning Oven              1
Name: Appliance, dtype: int64

In [31]:
# create sub-nodes from categorical column and join it to the existing df to create a new one
app_sub_nodes = pd.get_dummies(df_for_csv04.Appliance)
df_for_csv05 = pd.concat([df_for_csv04, app_sub_nodes], axis=1)
df_for_csv05.head()

,MlsId,MlsName,DateListed,StreetAddress,Price,Bedrooms,FullBathrooms,HalfBathrooms,Appliance,Appliances,...,Cooktop - Gas,Double Oven,Free Standing Gas,Gas,Microwave,Oven,Oven-Gas,Range Hood,RangeOven,Self Cleaning Oven
DateAsDate,,,,,,,,,,,,,,,,,,,,,
2016-11-18 07:45:06,316010702,ITECH,2016-11-18 07:45:06,"9378 Wilshire Boulevard, Suite 200",1025000.00,4,1,NaN,Built-In Gas,\n,...,0,0,0,0,0,0,0,0,0,0
2016-11-18 00:00:00,16181224,CLAW,2016-11-18 00:00:00,"11726 San Vicente Blvd, Suite 350",839000.00,2,2,1,RangeOven,\n,...,0,0,0,0,0,0,0,0,1,0
2016-11-18 00:00:00,316010634,ITECH,2016-11-18 00:00:00,"9378 Wilshire Boulevard, Suite 200",785000.00,2,3,NaN,Range Hood,\n,...,0,0,0,0,0,0,0,1,0,0
2016-11-18 00:00:00,16181228,CLAW,2016-11-18 00:00:00,"23410 Civic Center Way, C1",7488000.00,4,4,1,RangeOven,\n,...,0,0,0,0,0,0,0,0,1,0
2016-11-17 00:00:00,16180298,CLAW,2016-11-17 00:00:00,"11726 San Vicente Blvd, Suite 350",1125000.00,3,2,NaN,Self Cleaning Oven,\n,...,0,0,0,0,0,0,0,0,0,1


### Rooms... all sub-nodes comma joined

In [32]:
# this is a candidate for sub-nodes, but is redundant
df_for_csv05.Room.value_counts()

bathrooms    194
Name: Room, dtype: int64

In [33]:
# this is also a candidate for sub-nodes, but has redundant information, since Roomcount specifies
df_for_csv05.Rooms.value_counts()

\n\t            194
Name: Rooms, dtype: int64

In [34]:
# this is another candidate for sub-nodes, and contains relevant information
df_for_csv05.RoomCount.value_counts()

2    191
0     15
1      3
Name: RoomCount, dtype: int64

In [35]:
# this is also another candidate for sub-nodes, and contains relevant information
df_for_csv05.HalfBathrooms.value_counts()

1     71
2      5
3      2
99     1
Name: HalfBathrooms, dtype: int64

In [36]:
# this is clear candidate for sub-nodes, and contains relevant information
df_for_csv05.FullBathrooms.value_counts()

2    66
1    50
3    41
5    17
4    14
8     2
6     2
7     2
Name: FullBathrooms, dtype: int64

##### Create sub-nodes from `Roomcount`, `HalfBathrooms`, and `FullBathrooms` 
and join them to another iteration of the df

In [37]:
room_sub_nodes0 = pd.get_dummies(df_for_csv05.RoomCount, prefix='RoomCount')
room_sub_nodes1 = pd.get_dummies(df_for_csv05.HalfBathrooms, prefix='HalfBathrooms')
room_sub_nodes2 = pd.get_dummies(df_for_csv05.FullBathrooms, prefix='FullBathrooms')
room_sub_nodes3 = pd.get_dummies(df_for_csv05.Bedrooms, prefix='Bedrooms')


df_for_csv06 = pd.concat([df_for_csv05, room_sub_nodes0, room_sub_nodes1, room_sub_nodes2, room_sub_nodes3 ], axis=1)
df_for_csv06.head()

,MlsId,MlsName,DateListed,StreetAddress,Price,Bedrooms,FullBathrooms,HalfBathrooms,Appliance,Appliances,...,FullBathrooms_8,Bedrooms_0,Bedrooms_1,Bedrooms_2,Bedrooms_3,Bedrooms_4,Bedrooms_5,Bedrooms_6,Bedrooms_7,Bedrooms_8
DateAsDate,,,,,,,,,,,,,,,,,,,,,
2016-11-18 07:45:06,316010702,ITECH,2016-11-18 07:45:06,"9378 Wilshire Boulevard, Suite 200",1025000.00,4,1,NaN,Built-In Gas,\n,...,0,0,0,0,0,1,0,0,0,0
2016-11-18 00:00:00,16181224,CLAW,2016-11-18 00:00:00,"11726 San Vicente Blvd, Suite 350",839000.00,2,2,1,RangeOven,\n,...,0,0,0,1,0,0,0,0,0,0
2016-11-18 00:00:00,316010634,ITECH,2016-11-18 00:00:00,"9378 Wilshire Boulevard, Suite 200",785000.00,2,3,NaN,Range Hood,\n,...,0,0,0,1,0,0,0,0,0,0
2016-11-18 00:00:00,16181228,CLAW,2016-11-18 00:00:00,"23410 Civic Center Way, C1",7488000.00,4,4,1,RangeOven,\n,...,0,0,0,0,0,1,0,0,0,0
2016-11-17 00:00:00,16180298,CLAW,2016-11-17 00:00:00,"11726 San Vicente Blvd, Suite 350",1125000.00,3,2,NaN,Self Cleaning Oven,\n,...,0,0,0,0,1,0,0,0,0,0


In [38]:
for g in df_for_csv06.columns:
    print g

MlsId
MlsName
DateListed
StreetAddress
Price
Bedrooms
FullBathrooms
HalfBathrooms
Appliance
Appliances
Room
RoomCount
Rooms
Desc
Year
Month
Built-In And Free Standing
Built-In BBQ
Built-In Gas
Built-Ins
Cooktop - Gas
Double Oven
Free Standing Gas
Gas
Microwave
Oven
Oven-Gas
Range Hood
RangeOven
Self Cleaning Oven
RoomCount_0
RoomCount_1
RoomCount_2
HalfBathrooms_1
HalfBathrooms_2
HalfBathrooms_3
HalfBathrooms_99
FullBathrooms_1
FullBathrooms_2
FullBathrooms_3
FullBathrooms_4
FullBathrooms_5
FullBathrooms_6
FullBathrooms_7
FullBathrooms_8
Bedrooms_0
Bedrooms_1
Bedrooms_2
Bedrooms_3
Bedrooms_4
Bedrooms_5
Bedrooms_6
Bedrooms_7
Bedrooms_8


### All the columns we used to create the sub-nodes are now redundant, so we drop them
- Appliance
- RoomCount
- HalfBathrooms
- FullBathrooms
- Bedrooms

### The columns we labeled as redundant also get dropped
- Appliances
- Room
- Rooms


In [39]:
# make a list to facilitate programming
drop_list = ['Appliance',
'RoomCount',
'HalfBathrooms',
'FullBathrooms',
'Bedrooms',
'Appliances',
'Room',
'Rooms']
df_for_csv07 = df_for_csv06.drop(drop_list, axis=1)

In [40]:
df_for_csv07.columns

Index([u'MlsId', u'MlsName', u'DateListed', u'StreetAddress', u'Price',
       u'Desc', u'Year', u'Month', u'Built-In And Free Standing',
       u'Built-In BBQ', u'Built-In Gas', u'Built-Ins', u'Cooktop - Gas',
       u'Double Oven', u'Free Standing Gas', u'Gas', u'Microwave', u'Oven',
       u'Oven-Gas', u'Range Hood', u'RangeOven', u'Self Cleaning Oven',
       u'RoomCount_0', u'RoomCount_1', u'RoomCount_2', u'HalfBathrooms_1',
       u'HalfBathrooms_2', u'HalfBathrooms_3', u'HalfBathrooms_99',
       u'FullBathrooms_1', u'FullBathrooms_2', u'FullBathrooms_3',
       u'FullBathrooms_4', u'FullBathrooms_5', u'FullBathrooms_6',
       u'FullBathrooms_7', u'FullBathrooms_8', u'Bedrooms_0', u'Bedrooms_1',
       u'Bedrooms_2', u'Bedrooms_3', u'Bedrooms_4', u'Bedrooms_5',
       u'Bedrooms_6', u'Bedrooms_7', u'Bedrooms_8'],
      dtype='object')

In [41]:
for gg in df_for_csv07.columns:
    print gg

MlsId
MlsName
DateListed
StreetAddress
Price
Desc
Year
Month
Built-In And Free Standing
Built-In BBQ
Built-In Gas
Built-Ins
Cooktop - Gas
Double Oven
Free Standing Gas
Gas
Microwave
Oven
Oven-Gas
Range Hood
RangeOven
Self Cleaning Oven
RoomCount_0
RoomCount_1
RoomCount_2
HalfBathrooms_1
HalfBathrooms_2
HalfBathrooms_3
HalfBathrooms_99
FullBathrooms_1
FullBathrooms_2
FullBathrooms_3
FullBathrooms_4
FullBathrooms_5
FullBathrooms_6
FullBathrooms_7
FullBathrooms_8
Bedrooms_0
Bedrooms_1
Bedrooms_2
Bedrooms_3
Bedrooms_4
Bedrooms_5
Bedrooms_6
Bedrooms_7
Bedrooms_8


### Deliver the df as a `.csv` file

In [42]:
df_for_csv07.to_csv('BoojTestJG.csv', sep=',', index = False)

In [43]:
! ls

BoojCodeTest.ipynb BoojTestJG.csv
